In [34]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import csv
import os

In [36]:
cwd = os.getcwd()

In [3]:
start_date = [
    "2022-09-21", "2022-07-27", "2022-06-15", "2022-05-04", "2022-03-16", 
    "2018-12-19", "2018-09-26", "2018-06-13", "2018-03-21",
    "2017-12-13", "2017-06-14", "2017-03-15", "2016-12-14", "2015-12-16", "2006-06-29", "2006-05-10",
    "2006-03-28", "2006-01-31", "2005-12-13","2005-11-01", "2005-09-01","2005-09-20","2005-08-09","2005-06-30",
    "2005-05-03", "2005-03-22", "2005-02-02","2004-12-14","2004-11-10","2004-09-21","2004-08-10","2004-06-30",
    "2000-05-16","2000-03-21", "2000-02-02"
    ]
end_date = []

for i in range(len(start_date)): ## Generate End dates for each FOMC meeting term
    start_date[i] = datetime.strptime(start_date[i], "%Y-%m-%d") - timedelta(days = 1)
    end_date.append(start_date[i] + timedelta(days = 55))
    start_date[i] = datetime.strftime(start_date[i], "%Y-%m-%d") 
    end_date[i] = datetime.strftime(end_date[i], "%Y-%m-%d") # end_date and start_date as strings


In [111]:
def initTicker(stock):
    ticker = stock
    dir = cwd + "\\Data\\TransformedData\\{}.csv".format(ticker)
    df = pd.read_csv(dir)
    termInfo = pd.read_csv("TermInformation.csv")
    termInfo = termInfo.set_index('Unnamed: 0')
    termInfo.index.name = "Ticker"
    df.index.name = "Date"
    return df, termInfo

In [110]:
def getRecDays(termInfo, ticker):
    recDay1 = termInfo.loc[ticker][0]
    recDay2 = termInfo.loc[ticker][1]
    term = termInfo.loc[ticker][2]
    return recDay1, recDay2, term

In [109]:
def backtest(df, recDay1, recDay2, term):
    init = 1
    win = 0
    lose = 0
    for i in range(len(df)):
        vect = df.loc[len(df) - i - 1]
        vect = vect.drop('id')
        recDay1 = np.floor(recDay1).astype(int)
        recDay2 = np.floor(recDay2).astype(int)
        term = np.round(term).astype(int)
        buyPrice = 0.0
        sellPrice = 0.0
        if vect[recDay1] < vect[0]:
            buyPrice = vect[recDay1]
        else:
            print("{}th FOMC Return : 1.00".format(i))
            print("Cumulative Return : {}".format(init))
            continue
        for day in range(recDay1, recDay1 + term + 1):
            if vect[day] < 0.95 * buyPrice:
                sellPrice = vect[day]
                init = init * sellPrice / buyPrice
                lose = lose + 1
                break
        if sellPrice != 0:
            print("{}th FOMC : Buy {} Sell {} Return {}".format(i, buyPrice, sellPrice, sellPrice / buyPrice))
            print("Cumulative Return : {}".format(init))
            continue
        sellPrice = vect[recDay1 + term]
        init = init * sellPrice / buyPrice
        print("{}th FOMC : Buy {} Sell {} Return {}".format(i, buyPrice, sellPrice, sellPrice / buyPrice))
        print("Cumulative Return : {}".format(init))
        if(sellPrice > buyPrice):
            win = win + 1
    print(win, lose)

In [113]:
def pipeline(stock):
    df, termInfo = initTicker(stock)
    recDay1, recDay2, term = getRecDays(termInfo, stock)
    backtest(df, recDay1, recDay2, term)

In [119]:
pipeline("FLT")

0th FOMC Return : 1.00
Cumulative Return : 1
1th FOMC : Buy 143.92999267578125 Sell 150.25 Return 1.0439102872634427
Cumulative Return : 1.0439102872634427
2th FOMC : Buy 156.33999633789062 Sell 148.10000610351562 Return 0.9472944196789779
Cumulative Return : 0.988890389770138
3th FOMC : Buy 143.47999572753906 Sell 149.5800018310547 Return 1.0425146799913434
Cumulative Return : 1.03093274823773
4th FOMC Return : 1.00
Cumulative Return : 1.03093274823773
5th FOMC : Buy 199.1300048828125 Sell 201.2899932861328 Return 1.0108471267531554
Cumulative Return : 1.0421154064318434
6th FOMC Return : 1.00
Cumulative Return : 1.0421154064318434
7th FOMC : Buy 216.80999755859366 Sell 205.27999877929688 Return 0.9468198011663149
Cumulative Return : 0.9866955019101514
8th FOMC Return : 1.00
Cumulative Return : 0.9866955019101514
9th FOMC Return : 1.00
Cumulative Return : 0.9866955019101514
10th FOMC : Buy 226.9600067138672 Sell 247.3000030517578 Return 1.0896192973924856
Cumulative Return : 1.0751224